### Classification in Business

<center>
    <img src = 'images/uci_biz.png' />
</center>



------------


For this try-it, you are to explore some available datasets related to business applications of classification.  Using a dataset from the UCI Machine Learning Repository, scan datasets   under the subject area "BUSINESS" [link here](https://archive.ics.uci.edu/ml/datasets.php?format=&task=cla&att=&area=bus&numAtt=&numIns=&type=&sort=nameUp&view=table).  Find a dataset that looks interesting to you and decide how you could use Logistic Regression to help make a business decision using the data. 

In sharing your results, be sure to clearly describe the following:

- the dataset and its features
- the classification problem -- what are you classifying here?
- a business decision that can be supported using the results of the classification model

Share your summary on the appropriate discussion board for the activity. 

In [2]:
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsRegressor
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_transformer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import set_config

from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import ConfusionMatrixDisplay, RocCurveDisplay
from sklearn.linear_model import LogisticRegression
set_config("figure")

In [3]:
df=pd.read_excel("data/Absenteeism_at_work.xls")

In [4]:
df.head()

,ID,Reason for absence,Month of absence,Day of the week,Seasons,Transportation expense,Distance from Residence to Work,Service time,Age,Work load Average/day,...,Disciplinary failure,Education,Son,Social drinker,Social smoker,Pet,Weight,Height,Body mass index,Absenteeism time in hours
0,11,26,7,3,1,289,36,13,33,239554,...,0,1,2,1,0,1,90,172,30,4
1,36,0,7,3,1,118,13,18,50,239554,...,1,1,1,1,0,0,98,178,31,0
2,3,23,7,4,1,179,51,18,38,239554,...,0,1,0,1,0,0,89,170,31,2
3,7,7,7,5,1,279,5,14,39,239554,...,0,1,2,1,1,0,68,168,24,4
4,11,23,7,5,1,289,36,13,33,239554,...,0,1,2,1,0,1,90,172,30,2


In [5]:
df.dtypes

ID                                 int64
Reason for absence                 int64
Month of absence                   int64
Day of the week                    int64
Seasons                            int64
Transportation expense             int64
Distance from Residence to Work    int64
Service time                       int64
Age                                int64
Work load Average/day              int64
Hit target                         int64
Disciplinary failure               int64
Education                          int64
Son                                int64
Social drinker                     int64
Social smoker                      int64
Pet                                int64
Weight                             int64
Height                             int64
Body mass index                    int64
Absenteeism time in hours          int64
dtype: object

In [6]:

col         = 'Absenteeism time in hours'

df[col].describe()

count    740.000000
mean       6.924324
std       13.330998
min        0.000000
25%        2.000000
50%        3.000000
75%        8.000000
max      120.000000
Name: Absenteeism time in hours, dtype: float64

In [8]:

# conditions  = [ df[col] >= 40, (df[col] < 40) & (df[col]> 16), (df[col] <=16 ) & (df[col]> 8), (df[col] <=8) & (df[col]> 4) , (df[col] <=4) & (df[col]> 2), df[col] <=2]
# choices     = [5, 4, 3, 2, 1, 0 ]
conditions  = [ df[col] >= 40, df[col]< 40]
choices = [1, 0]
df["absence"] = np.select(conditions, choices, default=np.nan)
df["absence"]

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
735    0.0
736    0.0
737    0.0
738    0.0
739    0.0
Name: absence, Length: 740, dtype: float64

In [9]:

# y=np.where(df["Absenteeism time in hours"] > 40, 1,  0)
# y=pd.Series(y)
X_train, X_test, y_train, y_test = train_test_split(df.drop(["Absenteeism time in hours", "absence", "ID"], axis = 1), df['absence'], random_state = 442,
                                                   stratify = df['absence'])


In [10]:
X_train

,Reason for absence,Month of absence,Day of the week,Seasons,Transportation expense,Distance from Residence to Work,Service time,Age,Work load Average/day,Hit target,Disciplinary failure,Education,Son,Social drinker,Social smoker,Pet,Weight,Height,Body mass index
574,13,1,2,2,179,26,9,30,313532,96,0,3,0,0,0,0,56,171,19
131,27,1,4,2,118,10,10,37,308593,95,0,1,0,0,0,0,83,172,28
221,22,6,2,3,361,52,3,28,377550,94,0,1,1,1,0,4,80,172,27
331,12,11,6,4,291,31,12,40,284031,97,0,1,1,1,0,1,73,171,25
9,22,7,2,1,235,11,14,37,239554,97,0,3,1,0,0,1,88,172,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,13,3,6,2,260,50,11,36,343253,95,0,1,4,1,0,0,65,168,23
296,7,10,2,4,118,10,10,37,265017,88,0,1,0,0,0,0,83,172,28
77,26,10,4,4,289,36,13,33,253465,93,0,1,2,1,0,1,90,172,30
112,23,12,6,2,225,26,9,28,261306,97,0,1,1,0,0,2,69,169,24


In [11]:
list(X_train[1:1])

['Reason for absence',
 'Month of absence',
 'Day of the week',
 'Seasons',
 'Transportation expense',
 'Distance from Residence to Work',
 'Service time',
 'Age',
 'Work load Average/day ',
 'Hit target',
 'Disciplinary failure',
 'Education',
 'Son',
 'Social drinker',
 'Social smoker',
 'Pet',
 'Weight',
 'Height',
 'Body mass index']

In [12]:
selector = make_column_selector()
transformer = make_column_transformer((OneHotEncoder(handle_unknown="ignore"), selector),
                                   remainder = StandardScaler())


# transformer = make_column_transformer((OneHotEncoder(handle_unknown='ignore'), selector),
#                                      remainder =MinMaxScaler())


In [13]:
extractor = SelectFromModel(LogisticRegression(penalty='l2', solver = 'liblinear' ,random_state = 42))

In [14]:

from sklearn.impute import SimpleImputer
lgr_pipe = Pipeline([ ('transformer', transformer),
                    ('selector', extractor),
                    ('lgr', LogisticRegression(random_state=42, max_iter = 1000))])

lgr_pipe.fit(X_train, y_train)

pipe_1_acc = lgr_pipe.score(X_test, y_test)

In [15]:
pipe_1_acc

0.9675675675675676

In [16]:
print(y_test.value_counts(normalize = True))

0.0    0.972973
1.0    0.027027
Name: absence, dtype: float64


In [17]:
fp = 126
fn = 194
auc = 0.86

In [18]:
no_probs = lgr_pipe.predict_proba(X_test)[:, 0]
no_probs[:5]
high_prob_no = no_probs[no_probs > 0.95]


In [19]:
# YOUR CODE HERE
percent_of_test_data = len(high_prob_no)/len(y_test)
percent_of_no = len(high_prob_no)/sum(y_test ==0 )

### ANSWER CHECK
print(percent_of_test_data)
print(percent_of_no)

0.8486486486486486
0.8722222222222222


In [20]:
import sklearn

def get_feature_names(column_transformer):
    """Get feature names from all transformers.
    Returns
    -------
    feature_names : list of strings
        Names of the features produced by transform.
    """
    # Remove the internal helper function
    #check_is_fitted(column_transformer)
    
    # Turn loopkup into function for better handling with pipeline later
    def get_names(trans):
        # >> Original get_feature_names() method
        if trans == 'drop' or (
                hasattr(column, '__len__') and not len(column)):
            return []
        if trans == 'passthrough':
            if hasattr(column_transformer, '_df_columns'):
                if ((not isinstance(column, slice))
                        and all(isinstance(col, str) for col in column)):
                    return column
                else:
                    return column_transformer._df_columns[column]
            else:
                indices = np.arange(column_transformer._n_features)
                return ['x%d' % i for i in indices[column]]
        if not hasattr(trans, 'get_feature_names'):
        # >>> Change: Return input column names if no method avaiable
            # Turn error into a warning
            warnings.warn("Transformer %s (type %s) does not "
                                 "provide get_feature_names. "
                                 "Will return input column names if available"
                                 % (str(name), type(trans).__name__))
            # For transformers without a get_features_names method, use the input
            # names to the column transformer
            if column is None:
                return []
            else:
                return [name + "__" + f for f in column]

        return [name + "__" + f for f in trans.get_feature_names()]
    
    ### Start of processing
    feature_names = []
    
    # Allow transformers to be pipelines. Pipeline steps are named differently, so preprocessing is needed
    if type(column_transformer) == sklearn.pipeline.Pipeline:
        l_transformers = [(name, trans, None, None) for step, name, trans in column_transformer._iter()]
    else:
        # For column transformers, follow the original method
        l_transformers = list(column_transformer._iter(fitted=True))
    
    
    for name, trans, column, _ in l_transformers: 
        if type(trans) == sklearn.pipeline.Pipeline:
            # Recursive call on pipeline
            _names = get_feature_names(trans)
            # if pipeline has no transformer that returns names
            if len(_names)==0:
                _names = [name + "__" + f for f in column]
            feature_names.extend(_names)
        else:
            feature_names.extend(get_names(trans))
    
    return feature_names
    
feature_names=get_feature_names(transformer)
    

Pipeline(steps=[('transformer',
                 ColumnTransformer(remainder=StandardScaler(),
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(drop='first'),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x00000224BA8FC1C0>)])),
                ('selector',
                 SelectFromModel(estimator=LogisticRegression(penalty='l1',
                                                              random_state=42,
                                                              solver='liblinear'))),
                ('lgr', LogisticRegression(max_iter=1000, random_state=42))])

In [21]:
lgr_pipe.named_steps['lgr'].coef_[0]

array([-0.48619969,  0.24639905,  0.44848257,  0.7296261 , -0.27413751,
        0.34890643,  0.35498752,  1.64098637, -0.26334467, -0.20521153,
       -0.38415991,  1.53017734, -0.48918455, -1.12235554, -0.40859074,
       -0.41347601, -0.32216523, -1.02834766, -0.43427141, -0.3729636 ,
        0.32460192, -0.27685949,  0.5435602 , -0.08916182, -0.22765351,
        0.30257927, -0.32823611, -1.06852497, -0.13405496, -0.37614083,
       -0.09060808,  0.48423526, -0.06189387,  0.5515612 , -0.5286726 ,
       -0.3208967 ,  0.38001006, -0.22433922,  0.20365417,  0.52179982,
       -0.47130581,  0.50281444, -0.3129572 , -0.36129612, -0.25038972,
       -0.02997068,  0.26578324, -0.38240757, -0.29684453, -0.34161138,
        0.1890413 ,  0.5515612 ,  0.60045051,  0.45916193, -0.24277783,
       -0.25783208, -0.41346048,  0.37115415,  0.71728533, -0.22750872,
        0.7069961 ,  0.33194216, -0.48530719, -0.22250578, -0.30373152,
       -0.22095358, -0.23717479, -0.23639435,  0.38733293, -0.25

In [22]:
len(feature_names[0:102])

102

In [24]:
coef_df = pd.DataFrame({'feature': feature_names[0:103], 'coefs': lgr_pipe.named_steps['lgr'].coef_[0]})
coef_df['coefs'] = coef_df['coefs'].apply(abs)
coef_df = coef_df.sort_values(by = 'coefs', ascending = False)
coef_df.head(10)

,feature,coefs
7,onehotencoder__x0_8,1.640986
11,onehotencoder__x0_12,1.530177
13,onehotencoder__x0_14,1.122356
27,onehotencoder__x1_0,1.068525
17,onehotencoder__x0_18,1.028348
100,onehotencoder__x6_4,0.968984
3,onehotencoder__x0_4,0.729626
58,onehotencoder__x4_233,0.717285
60,onehotencoder__x4_246,0.706996
97,onehotencoder__x5_52,0.659682


From the abpve results it is clear that x0 and x1 columns are most important from onehotencoder analysis. These map to reason of absence and month of absence